In [1]:
from pydantic import BaseModel
from typing import Optional
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS, DCTERMS
from pyfuseki import FusekiUpdate
from rdflib.plugins.stores import sparqlstore
from rdflib.namespace import XSD
from datetime import datetime
from src.schemas.settings import Settings
from pysolr import Solr
from src.function.catalog.markeCreateSparql import MakeCreateSparql
from src.schemas.catalog.instance import BfInstance

In [15]:
url = "http://id.loc.gov/resources/items/21876543-050-15.rdf"
g = Graph()
g.parse(url)

<Graph identifier=Ne5bb87c69b2b4cfe9245f1658cb7e64a (<class 'rdflib.graph.Graph'>)>

In [16]:
g.serialize(destination='item_test.ttl')

<Graph identifier=Ne5bb87c69b2b4cfe9245f1658cb7e64a (<class 'rdflib.graph.Graph'>)>

ITEMS

In [6]:
req = {
    "itemOf": "instance#27",
    "instanceOf": "work#79",
    "items": [
        {
            "cdd": "001",
            "cutter": "P831c",
            "year": "1992",
            "collection": "Obras gerais",
            "shelf": "E1.P1",
            "barcode": "24-2",
            "adminMetadata": {
                "status": {
                    "value": "http://id.loc.gov/vocabulary/mstatus/n",
                    "label": "Novo"
                }
            }
        }
    ]
}

In [7]:
from src.schemas.catalog.items import Items_Schema
settings = Settings()
settings.base_url

'https://bibliokeia.com'

In [10]:
from src.schemas.adminMetadata import AdminMetadata
from typing import List


class Item(BaseModel):
    adminMetadata: AdminMetadata
    cdd: str
    cutter: str
    year: str
    collection: str
    shelf: str
    barcode: str

class Items_Schema(BaseModel):
  itemOf: str
  instanceOf: str
  items: List[Item]
  
request = Items_Schema(**req)
request

Items_Schema(itemOf='instance#27', instanceOf='work#79', items=[Item(adminMetadata=AdminMetadata(creationDate=None, status=BfElement(value='http://id.loc.gov/vocabulary/mstatus/n', label='Novo'), descriptionConventions=None, identifiedBy=None), cdd='001', cutter='P831c', year='1992', collection='Obras gerais', shelf='E1.P1', barcode='24-2')])

In [12]:
request.instanceOf

'work#79'

In [15]:
def DocItem(item, instance, work):

    item_id = f'item#{item.adminMetadata.identifiedBy}'


    doc = {
        "id": item_id,
        "type": "item",
        "cdd": item.cdd,
        "cutter": item.cutter,
        "year": item.year,
        "collection": item.collection,
        "shelf": item.shelf,
        "barcode": item.barcode,
        "itemOf": {'id': f'{item_id}/itemOf/{instance}' 
                   } 
        }
    
    item_doc = {
        "id": instance,
        "_root_": work,
        "hasItem": {"add": doc }
    }

    response = solr.add([item_doc], commit=True)
    return response

for item in request.items:
    item.adminMetadata.identifiedBy = 'teste'
    doc = DocItem(item, request.itemOf, request.instanceOf)


In [19]:
solr = Solr(f'{settings.solr}/solr/catalog/', timeout=10)

In [20]:
response = solr.add([doc], commit=True)

SolrError: Solr responded with an error (HTTP 400): [Reason: Exception writing document id instance#27 (root=work#79) to the index; possible analysis error: cannot change field "shelf" from index options=DOCS_AND_FREQS_AND_POSITIONS to inconsistent index options=DOCS]

In [14]:
request.model_dump()

{'itemOf': 'instance#27',
 'instanceOf': 'work#79',
 'items': [{'adminMetadata': {'creationDate': None,
    'status': {'value': 'http://id.loc.gov/vocabulary/mstatus/n',
     'label': 'Novo'},
    'descriptionConventions': None,
    'identifiedBy': None},
   'cdd': '001',
   'cutter': 'P831c',
   'year': '1992',
   'collection': 'Obras gerais',
   'shelf': 'E1.P1',
   'barcode': '24-2'}]}

In [ ]:
from src.function.catalog.bibframe.bfAdminMetadata import BfAdminMetadata


def MakeGraphItem(item, instance):
    BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
    identifier = f"{settings.base_url}/items/{item.adminMetadata.identifiedBy}"
    resource = URIRef(identifier)

    g = Graph(identifier=identifier)
    g.bind('bf', BF)
    g.add((resource, RDF.type, BF.Item))
    g = BfAdminMetadata(g, item.adminMetadata, resource, BF) 
    assigner_uri = URIRef(settings.organization_loc_uri)
    g.add((resource, BF.heldBy, assigner_uri))
    itemOf = URIRef(f"{settings.base_url}/instances/{instance}")
    g.add((resource, BF.itemOf, itemOf))
    # CDD
    shelfMark =  BNode()
    g.add((resource, BF.shelfMark, shelfMark))
    g.add((shelfMark, RDF.type, BF.ShelfMarkDdc))
    g.add((shelfMark, RDFS.label, Literal(item.cdd)))
    # Cutter
    bCutter = BNode()
    g.add((resource, BF.shelfMark, bCutter))
    g.add((bCutter, RDF.type, BF.Classification))
    g.add((bCutter, BF.classificationPortion, Literal(item.cutter)))
    # Barcode
    bNBarcode = BNode()
    g.add((resource, BF.identifiedBy, bNBarcode))
    g.add((bNBarcode, RDF.type, BF.Barcode))
    g.add((bNBarcode, RDF.value, Literal(item.barcode)))
    # shelf
    bNsublocation = BNode()
    g.add((resource, BF.sublocation, bNsublocation))
    g.add((bNsublocation, RDF.type, BF.Sublocation))
    g.add((bNsublocation, RDFS.label, Literal(item.shelf)))
    g.add((resource, DCTERMS.isPartOf, URIRef(f"{settings.base_url}/items")))

    graph = g.serialize(format='ttl')

    return graph

    
    # print(g.serialize(format='ttl'))


for item in request.items:
    now = datetime.now()
    item.adminMetadata.creationDate = now
    item.adminMetadata.identifiedBy = "teste"
    graph = MakeGraphItem(item, request.itemOf)
    sparql = MakeCreateSparql(graph, "teste", "items")
    

    print(sparql)

In [24]:
fuseki = FusekiUpdate("http://localhost:3030" , 'bk') 

In [27]:
response = fuseki.run_sparql(sparql) 
response

Instance 

In [14]:
settings = Settings()

In [34]:
req = {
    "instanceOf": {
        "value": "https://bibliokeia.com/works/77"
    },
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "resourceType": [
        {
            "value": "Instance",
            "label": "Instância"
        },
        {
            "value": "Print",
            "label": "Impresso"
        }
    ],
    "title": {
        "mainTitle": "Sol de feira"
    },
    "physicalDetails": {
        "carrier": {
            "value": "http://id.loc.gov/vocabulary/carriers/nc",
            "label": "Volume"
        },
        "extent": "124 p.",
        "issuance": {
            "value": "http://id.loc.gov/vocabulary/issuance/mono",
            "label": "Unidade única"
        },
        "media": {
            "value": "http://id.loc.gov/vocabulary/mediaTypes/n",
            "label": "Não mediado"
        }
    },
    "provisionActivity": [
        {
            "place": "Brasília",
            "agent": "Editora Universidade de Brasília",
            "date": "2012"
        }
    ],
    "editionStatement": {
        "value": "1. ed."
    },
    "responsibilityStatement": {
        "value": "Karl Raymund Popper"
    },
    "note": [
        {
            "value": "Este é um registro de teste"
        }
    ],
    "image": {
        "cover": "https://m.media-amazon.com/images/I/61vzOtbbfKL._SY466_.jpg"
    }
}

In [35]:
request = BfInstance(**req)
now = datetime.now()
request.adminMetadata.creationDate = now
request.adminMetadata.identifiedBy = "teste"
request.instanceOf.value

'https://bibliokeia.com/works/77'

In [36]:
request.instanceOf

BfElement(value='https://bibliokeia.com/works/77', label=None)

In [10]:
work_id = f'work#{request.instanceOf.value.split("/")[-1]}'
instance_id = f'instance#{request.adminMetadata.identifiedBy}'
instanceOf = {
    'id': f'{instance_id}/instanceOf/{work_id}', 
    'uri': request.instanceOf.value, 
    'label': request.instanceOf.label
    } 
instanceOf


{'id': 'instance#teste/instanceOf/work#76',
 'uri': 'https://bibliokeia.com/works/76',
 'label': None}

In [ ]:
def DocInstance(request):

    work_id = f'work#{request.instanceOf.value.split("/")[-1]}'
    instance_id = f'instance#{request.adminMetadata.identifiedBy}'
    provisionActivity = request.provisionActivity[0]
    instanceOf = {
    'id': f'{instance_id}/instanceOf/{work_id}', 
    'uri': request.instanceOf.value, 
    'label': request.instanceOf.label
    } 

    doc = {
        "id": instance_id,
        # "creationDate": request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S"),
        "type": [i.value for i in request.resourceType],
        "mainTitle": request.title.mainTitle,
        "subtitle": request.title.subtitle if request.title.subtitle != "" else None,
        "carrier": request.physicalDetails.carrier.label,
        "extent": request.physicalDetails.extent,
        "issuance": request.physicalDetails.issuance.label,
        "media": request.physicalDetails.media.label,
        "publicationAgent": provisionActivity.agent,
        "publicationDate": provisionActivity.date,
        "publicationPlace": provisionActivity.place,      
        # "serie": request.seriesStatement,
        "image": request.image.cover,
        "instanceOf": instanceOf
        }

    
    work = {
        "id": work_id,
        "hasInstance": {"add": doc }
    }

    response = solr.add([work], commit=True)
    return response

doc = DocInstance(request)
doc

In [38]:
solr = Solr(f'{settings.solr}/solr/catalog/', timeout=10)

In [21]:
response = solr.add([doc], commit=True)

In [ ]:
work = {
        "id": "work#77",
        "hasInstance": {"add": doc }
    }
work

In [48]:
response = solr.add([work], commit=True)

In [ ]:
work = {
        "id": work_id,
        "hasInstance": {"add": {
            'id': 'work#76',
            'mainTitle': 'Conjecturas e refutações'
            }}
    }
response = solr.add([work], commit=True)

In [33]:
work = {
        "id": work_id,
        "hasInstance": {
            "remove": {
            'id': 'work#76',
            'mainTitle': 'Conjecturas e refutações'
            }}
    }
response = solr.add([work], commit=True)

'work#76'

In [67]:
class BfElement(BaseModel):
    value: str
    label: Optional[str] = None
    
class AdminMetadata(BaseModel):
    creationDate: datetime = None
    status: BfElement
    descriptionConventions: BfElement
    identifiedBy: Optional[str] = None

class Title(BaseModel):
    mainTitle: str
    subtitle: Optional[str] = None

class PhysicalDetails(BaseModel):
    carrier: BfElement
    extent: str
    issuance: BfElement
    media: BfElement

class ProvisionActivity(BaseModel):
    place: str
    agent: str
    date: str    

class EditionStatement(BaseModel):
    value: str

class BfInstance(BaseModel):
    resourceType: list[BfElement]
    adminMetadata: AdminMetadata
    title: Title
    variantTitle: Optional[list[Title]] = None
    physicalDetails: PhysicalDetails
    provisionActivity: list[ProvisionActivity]
    editionStatement: BfElement
    responsibilityStatement: BfElement
    note: list[BfElement]
    instanceOf: BfElement

request = BfInstance(**req)


In [40]:
def BFPhysicalDetails(g, resource, physicalDetails, BF):
    carrier = URIRef(physicalDetails.carrier.value)
    g.add((carrier, RDF.type, BF.Carrier))
    g.add((carrier, RDFS.label, Literal(physicalDetails.carrier.label)))
    g.add((resource, BF.carrier, carrier))

    extent = BNode()
    g.add((resource, BF.extent, extent))
    g.add((extent, RDF.type, BF.Extent))
    g.add((extent, RDFS.label, Literal(physicalDetails.extent)))

    media = URIRef(physicalDetails.media.value)
    g.add((media, RDF.type, BF.Media))
    g.add((media, RDFS.label, Literal(physicalDetails.media.label)))
    g.add((resource, BF.media, media))
    
    return g


In [54]:
def BFNote(g, resource, notes, BF):
    for note in notes:
        bnNote = BNode()
        g.add((resource, BF.note, bnNote))
        g.add((bnNote, RDF.type, BF.Note))
        g.add((bnNote, RDFS.label, Literal(note.value)))
    return g
        


In [57]:
def BfProvisionActivity(g, resource, provisionActivity, BF):
    for i in provisionActivity:
        bnProvisionActivity = BNode()
        g.add((bnProvisionActivity, RDF.type, BF.ProvisionActivity))
        g.add((bnProvisionActivity, RDF.type, BF.Publication))
        # Editor
        bnAgent = BNode()
        g.add((bnProvisionActivity, BF.agent, bnAgent))
        g.add((bnAgent, RDF.type, BF.Agent))
        g.add((bnAgent, RDFS.label, Literal(i.agent)))
        # Date
        g.add((bnProvisionActivity, BF.date, Literal(i.date)))
        # Place
        bnPlace = BNode()
        g.add((bnProvisionActivity, BF.place, bnPlace))
        g.add((bnPlace, RDF.type, BF.Place))
        g.add((bnPlace, RDFS.label, Literal(i.place)))
        g.add((resource, BF.provisionActivity, bnProvisionActivity))
    
    return g

In [61]:
from src.function.catalog.bibframe.bfTitle import BfTitle
from src.function.catalog.bibframe.bfAdminMetadata import BfAdminMetadata
from src.function.catalog.bibframe.bfType import BfType


def MakeGraphInstance(request):

    BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
    identifier = f"https://bibliokeia/works/{request.adminMetadata.identifiedBy}"
    resource = URIRef(identifier)

    g = Graph(identifier=identifier)
    g.bind('bf', BF)
    
    # resourceType
    BfType(g, resource, request.resourceType)
    # AdminMetadata
    g = BfAdminMetadata(g, request.adminMetadata, resource, BF) 
    # Title
    g = BfTitle(g, request, resource, BF)  
    
    # PhysicalDetails
    g = BFPhysicalDetails(g, resource, request.physicalDetails, BF)
    g = BFNote(g, resource, request.note, BF)
    g = BFProvisionActivity(g, resource, request.provisionActivity, BF)
    g.add((resource, DCTERMS.isPartOf, URIRef("http://bibliokeia/resources/instances")))

 
    graph = g.serialize(format='ttl')
    # g.serialize(destination="instance_test.ttl", format='ttl')

    return graph

graph = MakeGraphInstance(request)

In [65]:
sparql = MakeCreateSparql(graph, "teste", "instance")
print(sparql)


        PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        INSERT DATA {
            GRAPH <https://bibliokeia.com/instance/teste> 
            {
                <https://bibliokeia/works/teste> a bf:Instance,
        bf:Print ;
    bf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <http://id.loc.gov/vocabulary/organizations/brmninpa> ;
            bf:creationDate "2024-03-19"^^xsd:date ;
            bf:descriptionConventions <http://id.loc.gov/vocabulary/descriptionConventions/aacr> ;
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "BiblioKeia" ;
                    bf:generationDate "2024-03-19T09:33:41"^^xsd:dateTime ] ;
            bf:identifiedBy [ a bf:Local ;
                    bf:assigner <http://id.loc.gov/vocabulary/organizati

In [17]:
class ResourceProperties(BaseModel):
    content: BfElement
    genreForm: Optional[BfElement] = None

class Contribution(BaseModel):
    term: BfElement
    role: BfElement

class Subject(BaseModel):
    type: str
    lang: str
    term: BfElement

class Classification(BaseModel):
    cdd: str
    cutter: str

class BfWork(BaseModel):
    resourceType: list[BfElement]
    adminMetadata: AdminMetadata
    title: Title
    variantTitle: Optional[list[Title]] = None
    language: list[BfElement]
    genreForm: list[BfElement] = None
    contribution: Optional[list[Contribution]] = None
    subject: Optional[list[Subject]] = None
    classification: Classification
    note: Optional[str] = None
    summary: Optional[str] = None
    tableOfContents: Optional[str] = None



In [18]:
req = {
    "resourceType": [
        {
            "value": "Work",
            "label": "Obra"
        },
        {
            "value": "Text",
            "label": "Texto"
        }
    ],
    "adminMetadata": {
        "status": {
            "value": "http://id.loc.gov/vocabulary/mstatus/n",
            "label": "Novo"
        },
        "descriptionConventions": {
            "value": "http://id.loc.gov/vocabulary/descriptionConventions/aacr",
            "label": "AACr"
        }
    },
    "classification": {
        "cdd": "001",
        "cutter": "P831c"
    },
    "title": {
        "mainTitle": "Conjecturas e refutações"
    },
    "language": [
        {
            "value": "por",
            "label": "Português"
        }
    ],
    "contribution": [
        {
            "term": {
                "value": "https://bibliokeia.com/authority/PersonalName/10",
                "label": "Popper, Karl R."
            },
            "role": {
                "value": "http://id.loc.gov/vocabulary/relators/aut",
                "label": "Autor"
            }
        },
        {
            "term": {
                "value": "https://bibliokeia.com/authority/PersonalName/12",
                "label": "Bath, Sérgio"
            },
            "role": {
                "value": "http://id.loc.gov/vocabulary/relators/trl",
                "label": "Tradutor"
            }
        }
    ],
    "subject": [
        {
            "term": {
                "value": "https://bibliokeia.com/authority/Topic/11",
                "label": "Metodologia"
            },
            "lang": "por",
            "type": "Topic"
        }
    ]
}
request = BfWork(**req)

In [21]:
request.adminMetadata.creationDate

In [4]:
settings = Settings()

In [22]:


def BfGenreForm(g, resource, genreForm, BF):
    for i in genreForm:
        uri = URIRef(i.value)
        g.add((uri, RDF.type, BF.GenreForm ))
        g.add((uri, RDFS.label, Literal(i.label)))
        g.add((resource, BF.genreForm, uri))
    return g


def BfAdminMetadata(g, adminMetadata, resource, BF):
    now = datetime.now()  

    bNadminMetadata = BNode()
    g.add((resource, BF.adminMetadata, bNadminMetadata))
    g.add((bNadminMetadata, RDF.type, BF.AdminMetadata ))
    # assigner
    assigner_uri = URIRef(settings.organization_loc_uri)
    g.add((assigner_uri, RDF.type, BF.Organization))
    g.add((assigner_uri, RDFS.label, Literal(settings.organization)))
    g.add((bNadminMetadata, BF.assigner, assigner_uri ))
    # creationDate
    formatted_date = now.strftime("%Y-%m-%d")
    g.add((bNadminMetadata, BF.creationDate, Literal(formatted_date, datatype=XSD.date) ))
    # descriptionConventions
    descriptionConventions = URIRef(adminMetadata.descriptionConventions.value)
    g.add((descriptionConventions, RDF.type, BF.DescriptionConventions ))
    g.add((descriptionConventions, RDFS.label, Literal(adminMetadata.descriptionConventions.label) ))
    g.add((bNadminMetadata, BF.descriptionConventions, descriptionConventions ))
    # generationProcess
    formatted_dateTime = now.strftime("%Y-%m-%dT%H:%M:%S")
    generationProcess = BNode() 
    g.add((bNadminMetadata, BF.generationProcess, generationProcess))
    g.add((generationProcess, RDF.type, BF.GenerationProcess))
    g.add((generationProcess, RDFS.label, Literal(settings.app_name) ))
    g.add((generationProcess, BF.generationDate, Literal(formatted_dateTime, datatype=XSD.dateTime) ))
    # identifiedBy
    identifiedBy =  BNode() 
    g.add((bNadminMetadata, BF.identifiedBy, identifiedBy))
    g.add((identifiedBy, RDF.type, BF.Local))
    g.add((identifiedBy, BF.assigner, assigner_uri))
    g.add((identifiedBy, RDF.value, Literal(adminMetadata.identifiedBy)))
    # status
    status = URIRef(adminMetadata.status.value)
    g.add((status, RDF.type, BF.Status))
    g.add((status, RDFS.label, Literal(adminMetadata.status.label)))
    g.add((bNadminMetadata, BF.status, status))

    return g

def BfTitle(g, resource, BF):
    title = BNode()
    g.add((resource, BF.title, title))
    g.add((title, RDF.type, BF.Title))
    g.add((title, BF.mainTitle, Literal(title.mainTitle)))
    if request.title.subtitle:
        g.add((title, BF.subtitle, Literal(request.title.subtitle)))
    if request.variantTitle:
        for i in request.variantTitle:
            variantTitle = BNode()
            g.add((resource, BF.title, variantTitle))
            g.add((variantTitle, RDF.type, BF.VariantTitle))
            g.add((variantTitle, BF.mainTitle, Literal(i.mainTitle)))
            if i.subtitle:
                g.add((variantTitle, BF.subtitle, Literal(i.subtitle)))

    return g

def BFLanguage(g, resource, BF):
    for i in request.language:
        language = URIRef(f"http://id.loc.gov/vocabulary/languages/{i.value}")
        g.add((language, RDF.type, BF.Language))
        g.add((language, RDFS.label, Literal(i.label)))
        g.add((resource, BF.language, language ))
    return g  

def BFContribution(g, resource, BF):
    for i in request.contribution:
        contribution = BNode()
        g.add((resource, BF.contribution, contribution))
        g.add((contribution, RDF.type, BF.Contribution))
        agent = URIRef(i.term.value)
        g.add((agent, RDF.type, BF.Agent))
        g.add((agent, RDF.type, BF.Person))
        g.add((agent, RDFS.label, Literal(i.term.label)))
        g.add((contribution, BF.agent, agent))
        role = URIRef(i.role.value)
        g.add((role, RDF.type, BF.Role))
        g.add((role, RDFS.label, Literal(i.role.label)))
        g.add((contribution, BF.role, role))

    return g

def BFSubject(g, resource, BF):
    for i in request.subject:
        subject = URIRef(i.term.value)
        g.add((subject, RDF.type, BF.Topic))
        g.add((subject, RDFS.label, Literal(i.term.label, lang=i.lang)))
        g.add((resource, BF.subject, subject))

    return g

def BfClassification(g, resource, classification, BF):
    cdd = BNode()
    g.add((resource, BF.classification, cdd))
    g.add((cdd, RDF.type, BF.ClassificationDdc))
    g.add((cdd, BF.classificationPortion, Literal(classification.cdd)))
    cutter = BNode()
    g.add((resource, BF.classification, cutter))
    g.add((cutter, RDF.type, BF.Classification))
    g.add((cutter, BF.classificationPortion, Literal(classification.cutter)))
    return g



In [ ]:
def MakeGraphWork(request):

    BF = Namespace("http://id.loc.gov/ontologies/bibframe/")
    identifier = f"https://bibliokeia/works/{request.adminMetadata.identifiedBy}"
    resource = URIRef(identifier)

    g = Graph(identifier=identifier)
    g.bind('bf', BF)
    
    # resourceType
    BfType(g, resource, request.resourceType)
    # AdminMetadata
    g = BfAdminMetadata(g, request.adminMetadata, request, resource, BF) 
    # BfClassification
    g = BfClassification(g, resource, request.classification, BF)
    # Title
    g = BfTitle(g, request, resource, BF)
    # Language
    g = BFLanguage(g, request.language, resource, BF)
    # Contribution
    g = BFContribution(g, request.contribution, resource, BF)
    # Subject
    g = BFSubject(g, request.subject, resource, BF) 
    # GenreForm
    if request.genreForm and request.genreForm.valeu != "":
        g = BfGenreForm(g, resource, request.genreForm, BF)    
 
    graph = g.serialize(format='ttl')
    # g.serialize(destination="work_test.ttl", format='ttl')

    return graph

def MakeSparql(graph, ):
    graph = graph.split("\n\n")
    graph = "\n\n".join(graph[1:])

    graph = f"""
        PREFIX bf: <http://id.loc.gov/ontologies/bibframe/>  
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

        INSERT DATA {{
            GRAPH <https://bibliokeia/works/10> 
            {{
                {graph} }} }} """

    return graph


graph = MakeGraphWork(request)
sparql = MakeSparql(graph)
print(sparql)

In [27]:
fuseki = FusekiUpdate("http://localhost:3030" , 'bk') 
response = fuseki.run_sparql(sparql) 
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [50]:
# url = 'http://localhost:3030/bk/data?graph=https:%2F%2Fbibliokeia%2Fworks%2F10'
url = "http://id.loc.gov/resources/works/21984730.rdf"
g = Graph()
g.parse(url)

<Graph identifier=Nf8a4447e118c409ab437162e76a2afcd (<class 'rdflib.graph.Graph'>)>

In [37]:
def DocWork(request, work_id):

    work_id = f'work#{work_id}'

    doc = {
        "id": work_id,
        "creationDate": request.adminMetadata.creationDate.strftime("%Y-%m-%dT%H:%M:%S"),
        "type": [i.value for i in request.resourceType],
        # "content": request.content.label,
        "mainTitle": request.title.mainTitle,
        'language': [i.label for i in request.language],
        "subtitle": request.title.subtitle,
        "cdd": request.classification.cdd,
        "cutter": request.classification.cutter,
        "note": request.note,
        "summary": request.summary,
        "tableOfContents": request.tableOfContents,
        # "supplementaryContent": [i.label for i in request.supplementaryContent] if request.supplementaryContent else None,
        # "illustrativeContent": [i.label for i in request.illustrativeContent] if request.illustrativeContent else None,
        # "intendedAudience": [i.label for i in request.intendedAudience] if request.intendedAudience else None,
        # "geographicCoverage": [i.label for i in request.geographicCoverage] if request.geographicCoverage else None,
        "isPartOf": "Work"

    }
    # contribution
    if request.contribution:
        contributions = list()
        for i in request.contribution:
            c = {"id": f"{work_id}/contribution/{i.term.value.split('/')[-1]}",
                 "agent": i.term.value,
                 "label": i.term.label,
                 "role": i.role.value,
                 "roleLabel": i.role.label}
            contributions.append(c)
        doc['contribution'] = contributions

    # subject
    if request.subject:
        subjects = list()
        for i in request.subject:
            s = {"id": f"{work_id}/subject/{i.term.value.split('/')[-1]}",
                 "type": i.type,
                 "uri": i.term.value,
                 "label": i.term.label}
            subjects.append(s)
        doc['subject'] = subjects

    if request.genreForm:
        pass

    responseSolr = solr.add([doc], commit=True)

    return responseSolr

doc = DocWork(request, "teste")

DOC: {'id': 'work#teste', 'creationDate': '2024-03-14T10:35:59', 'type': ['Work', 'Text'], 'mainTitle': 'Conjecturas e refutações', 'language': ['Português'], 'subtitle': None, 'cdd': '001', 'cutter': 'P831c', 'note': None, 'summary': None, 'tableOfContents': None, 'isPartOf': 'Work', 'contribution': [{'id': 'work#teste/contribution/10', 'agent': 'https://bibliokeia.com/authority/PersonalName/10', 'label': 'Popper, Karl R.', 'role': 'http://id.loc.gov/vocabulary/relators/aut', 'roleLabel': 'Autor'}, {'id': 'work#teste/contribution/12', 'agent': 'https://bibliokeia.com/authority/PersonalName/12', 'label': 'Bath, Sérgio', 'role': 'http://id.loc.gov/vocabulary/relators/trl', 'roleLabel': 'Tradutor'}], 'subject': [{'id': 'work#teste/subject/11', 'type': 'Topic', 'uri': 'https://bibliokeia.com/authority/Topic/11', 'label': 'Metodologia'}]}


In [39]:
solr = Solr(f'{settings.solr}/solr/catalog/', timeout=10)

In [40]:
responseSolr = solr.add([doc], commit=True)
responseSolr

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":82\n  }\n}'

In [13]:
request.adminMetadata.creationDate

'08/03/2024'